## Debug the regression model

In [13]:
cancer = '/Users/wangmeng/Documents/Research/Gentles/repos/MethylationPrediction/data/results/LAML/'
methylation = '/Users/wangmeng/Documents/Research/Gentles/repos/MethylationPrediction/data/processed/HM27.RData'
rnaseq = '/Users/wangmeng/Documents/Research/Gentles/repos/MethylationPrediction/data/processed/RNAseq.RData'
convert2M = as.numeric('0')

In [2]:
load(methylation)
load(rnaseq)

In [5]:
source('../R/regressionModel/regressionModel.R')
source('../R/correlation_HM450/computeCorrelation.R')

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: grid
ComplexHeatmap version 1.17.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://bioconductor.org/packages/ComplexHeatmap/

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

Loading required package: viridisLite


In [26]:
CGI.probes <- read.csv('/Users/wangmeng/Documents/Research/Gentles/repos/MethylationPrediction/data/promoterRegionProbes/CGICorFdr10.tsv',
                      sep='\t', header=T, stringsAsFactors=F)
probe.id <- as.character(CGI.probes[,1])
subsetProbes <- probe.id
center <- T
scale <- T
convert2M <- as.logical(convert2M)
percent.test <- 0.2
cutoff <- 20
alpha <- 1
subset.method <- 'vbsr'
distance <- 'euclidean'
method <- 'ward.D2'
dir.create(cancer, showWarnings=F)
imagefolder <- paste0('vbsr-', ifelse(convert2M, 'M', 'B'), '-plots/')
dir.create(file.path(cancer, imagefolder), showWarnings=F)
imagefolder <- paste0(cancer, imagefolder)
datafolder <- paste0('vbsr-', ifelse(convert2M, 'M', 'B'), '-data/')
dir.create(file.path(cancer, datafolder), showWarnings=F)
datafolder <- paste0(cancer, datafolder)

In [27]:
# preprocess and hclust
methylation <- data.HM27
rnaseq.processed <- PreprocessRNAseq(RNAseq)
rownames(rnaseq.processed$rnaseq) <- rnaseq.processed$g.geneid[,2]
methylation.processed <- PreprocessMethylation(methylation, convert2M=convert2M,
                                                 subsetProbes=subsetProbes)
processed <- matchColumns(rnaseq.processed$rnaseq,
                            methylation.processed$methylation)
methylation <- processed$methylation
rnaseq <- processed$rnaseq
methylprobes.geneid <- methylation.processed$m.geneid[,2]
methylation <- AverageMethylation(methylation, methylprobes.geneid)
methylation <- data.matrix(methylation[,-1])
rownames(methylation) <- methylation.processed$m.geneid[!duplicated(methylprobes.geneid), 1]
test.idx <- sample(seq(dim(methylation)[2]), round(dim(methylation)[2]*percent.test))
clustering <- ClusterMethylation(methylation, distance=distance, method=method, cutoff=cutoff)
cluster <- clustering$cluster
avemethyl <- AverageMethylation(methylation, cluster)

In [28]:
i <- 16
model <- RegressMethylationOnRNAseq(avemethyl[i, -1], rnaseq, test.idx, method=subset.method)

In [29]:
coef <- ExtractVBSRSignificantCoef(model, m=dim(rnaseq)[1], coef.names=unlist(rnaseq.processed$g.geneid[,2]))

In [30]:
# only 1 probe ends up in the cluster ...
base.name <- paste0('cluster', i)
path.fig <- paste0(imagefolder, '/', base.name, '.eps')
ordering <- order(abs(coef[-1,2]), decreasing=T)
ordering.genes <- as.character(coef[-1,1][ordering])
ModuleHeatmap(t(data.matrix(methylation[cluster==i, -test.idx])),
                  t(data.matrix(avemethyl[i, -1][,-test.idx])),
                  t(rnaseq[ordering.genes, -test.idx]),
                  path.fig, center=center, scale=scale)

`nrow` of all heatmaps:
        methylation Average methylation 
                  1                 136 


ERROR: Error in .local(object, ...): `nrow` of all heatmaps should be the same.


In [25]:
table(cluster)

cluster
  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
 41 208  35   6  40  77  20  24  13  16  26  13  20  16   8   1   1  18   5   2 

In [ ]:
# run the function as a whole
RunModel(methylation, RNAseq, imagefolder, datafolder, convert2M=convert2M, subsetProbes=subsetProbes, distance=distance, method=method,
         cutoff=cutoff, percent.test=percent.test, alpha=alpha, center=center, scale=scale, subset.method=subset.method)